In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sqlalchemy import create_engine, inspect
from sqlalchemy.orm import Session
from sqlalchemy.ext.automap import automap_base
from datetime import datetime, timedelta


In [ ]:
# Step 2: Connect to SQLite database
engine = create_engine('sqlite:///hawaii.sqlite')

In [ ]:
# Step 3: Reflect tables into classes
Base = automap_base()
Base.prepare(engine, reflect=True)
Station = Base.classes.station
Measurement = Base.classes.measurement

In [ ]:
# Step 4: Create a session
session = Session(engine)

In [ ]:
# Precipitation Analysis

# Step 1: Find the most recent date in the dataset
most_recent_date = session.query(Measurement.date).order_by(Measurement.date.desc()).first()
most_recent_date = datetime.strptime(most_recent_date[0], '%Y-%m-%d').date()

# Step 2: Calculate the date 1 year ago from the most recent date
one_year_ago = most_recent_date - timedelta(days=365)

# Step 3: Query the previous 12 months of precipitation data
precipitation_data = session.query(Measurement.date, Measurement.prcp).\
    filter(Measurement.date >= one_year_ago).all()

# Step 4: Load query results into a Pandas DataFrame
df_precipitation = pd.DataFrame(precipitation_data, columns=['date', 'prcp'])

# Step 5: Sort the DataFrame values by "date"
df_precipitation = df_precipitation.sort_values('date')

# Step 6: Plot the results
df_precipitation.plot(x='date', y='prcp', rot=90)
plt.xlabel('Date')
plt.ylabel('Precipitation (inches)')
plt.title('Precipitation Analysis (12 months)')
plt.tight_layout()
plt.show()


In [ ]:
# Station Analysis

# Query the total number of stations
station_count = session.query(func.count(Station.station)).scalar()
print(f'Total number of stations: {station_count}')

In [ ]:
# Query the most active stations
active_stations = session.query(Measurement.station, func.count(Measurement.station)).\
    group_by(Measurement.station).order_by(func.count(Measurement.station).desc()).all()

print('Most active stations:')
for station, count in active_stations:
    print(f'Station: {station}, Observations: {count}')

In [ ]:
# Get the station with the highest number of observations
most_active_station = active_stations[0][0]

# Query the last 12 months of temperature observation data for the most active station
temperature_data = session.query(Measurement.date, Measurement.tobs).\
    filter(Measurement.station == most_active_station).\
    filter(Measurement.date >= one_year_ago).all()


In [ ]:
# Load query results into a Pandas DataFrame
df_temperature = pd.DataFrame(temperature_data, columns=['date', 'tobs'])

In [ ]:
# Plot the results as a histogram
df_temperature.plot.hist(bins=12)
plt.xlabel('Temperature (°F)')
plt.ylabel('Frequency')
plt.title('Temperature Observation Analysis (12 months)')
plt.tight_layout()
plt.show()

In [ ]:
# Close the session
session.close()
